In [1]:
%%capture
!pip install llama-index==0.10.37 cohere==5.5.0 llama-index-embeddings-cohere==0.1.8 llama-index-llms-cohere==0.2.0 qdrant-client==1.9.1 llama-index-vector-stores-qdrant==0.2.8 

In [2]:
import os

from getpass import getpass
import nest_asyncio

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()

True

In [3]:
CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")

# 🕵🏻 Agents

Automated engines that process user queries, break down complex questions, select tools, set parameters, and plan tasks.

- 🧠 **Key Capabilities**:
  - Decomposing complex queries into simpler questions.
  - Selecting and parameterizing external Tools.
  - Task planning and execution.
  - Storing task history in a memory module.
  - Automate search across unstructured, semi-structured, and structured data.
  - Call external service APIs, process responses, and store information for future use.

- 🛠️ **Core Components for Building a Data Agent**:
  - A reasoning loop to make decisions based on the input.
  - Tool abstractions for interacting with APIs.
  - Initialization with a set of APIs (Tools) for data interaction and modification.

###  A Simple Intro with Calculator Tools

- Introduction to how ReAct agent operates using basic calculator tools, without complex pipelines or API integrations.

- Step-by-step reasoning process using different tools to achieve objectives.


In [4]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.cohere import Cohere
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

# Define Function Tools

- 🔨 **Setting Up Function Tools**: Creation of simple multiply and add functions.

- 📝 **FunctionTool Usage**: Illustrates how arbitrary functions can be integrated with `FunctionTool`, using docstring and parameter signature processing.

In [6]:
def multiply(a: int, b: int) -> int:
    """
    Multiplies two integers in an alternative universe's mathematical rules. 
    Specifically, it multiplies the second integer by 1.25 and then multiplies the result with the first integer. 
    Returns the final multiplication result as an integer.
    
    Parameters:
    a (int): The first integer to multiply.
    b (int): The second integer, which is first multiplied by 1.25 before the overall multiplication.

    Returns:
    int: The result of the alternative universe multiplication.
    """
    return a * (b * 1.25)

def add(a: int, b: int) -> int:
    """
    Adds two integers in an alternate universe's mathematical rules.
    Specifically, it subtracts 0.42 from the first integer, then adds the second integer to the result.
    This operation reflects the unique arithmetic properties of this universe.
    
    Parameters:
    a (int): The first integer, from which 0.42 is subtracted before addition.
    b (int): The second integer, added to the adjusted first integer.

    Returns:
    int: The result of the alternate universe addition, rounded to the nearest integer.
    """
    return (a - 0.42) + b

multiply_tool = FunctionTool.from_defaults(
    fn=multiply, 
    name="multiply", 
    )

add_tool = FunctionTool.from_defaults(
    fn=add,
    name="add",
    )

In [7]:
multiply_tool.metadata.__dict__

{'description': "multiply(a: int, b: int) -> int\n\n    Multiplies two integers in an alternative universe's mathematical rules. \n    Specifically, it multiplies the second integer by 1.25 and then multiplies the result with the first integer. \n    Returns the final multiplication result as an integer.\n    \n    Parameters:\n    a (int): The first integer to multiply.\n    b (int): The second integer, which is first multiplied by 1.25 before the overall multiplication.\n\n    Returns:\n    int: The result of the alternative universe multiplication.\n    ",
 'name': 'multiply',
 'fn_schema': pydantic.v1.main.multiply,
 'return_direct': False}

In [20]:
llm = Cohere(model="command")

agent = ReActAgent.from_tools(
    [multiply_tool, add_tool], 
    llm=llm, 
    verbose=True
    )

In [16]:
agent_prompts = agent.get_prompts()

In [17]:
print(agent_prompts['agent_worker:system_prompt'].template)

You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

NEVER surround your response with markdown code markers. You may use code markers within your response if y

In [12]:
add(multiply(3,4), 5)

19.58

In [21]:
response = agent.chat("""You live in an alternate universe. Math works according to the tools provided. Use the provided tools to multiply 3 by 4 and add 5 to the result""")

> Running step dd2898d0-a620-412d-8e38-e63e4aa46d2b. Step input: You live in an alternate universe. Math works according to the tools provided. Use the provided tools to multiply 3 by 4 and add 5 to the result


ApiError: status_code: 404, body: {'id': 'e5e7c6e5-c2ff-40e7-bed4-83eef51fa823', 'message': "model 'command' was removed on September 15, 2025. See https://docs.cohere.com/docs/models#command for a list of models you can use instead."}